In [2]:
%pip install -r ../../requirements.txt

In [2]:
import os 
import sys
import json
from types import SimpleNamespace

import torch
import torchaudio
import torch.nn as nn
import numpy as np
import pandas as pd
import lightning as L
from tqdm import tqdm
from timm import create_model, list_models
from torch.utils.data import DataLoader
from lightning.pytorch.loggers import CSVLogger, TensorBoardLogger
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor
from lightning.pytorch.tuner import Tuner

sys.path.append('../../src')

from custom.data import AudioDataset, DataModule
from custom.trainer import TrainModule
from custom.net import SimpleCNN
from custom.utils import batch_to_device

%load_ext autoreload
%autoreload 2

In [3]:
cfg = SimpleNamespace()
cfg.n_classes = 66
cfg.pretrained = True
cfg.backbone = 'tf_efficientnetv2_s.in21k'
cfg.in_chans = 1
cfg.lr = 3e-4
cfg.batch_size = 32

cfg.wav_crop_len = 15
cfg.sample_rate = 44100
cfg.n_mels = 128
cfg.n_fft = 2048
cfg.fmin = 300
cfg.fmax = cfg.sample_rate / 2 
cfg.window_size = cfg.n_fft
cfg.hop_length = int(cfg.n_fft / 2)
cfg.power = 2
cfg.top_db = 80.0

In [4]:
class_weights = np.load('class_weights.npy')

In [5]:
class_weights.shape

In [6]:
tb_logger = TensorBoardLogger(save_dir="./", version=1, name="effnet_baseline")
lr_monitor = LearningRateMonitor(logging_interval='epoch')
checkpoint_callback = ModelCheckpoint(monitor="val_f1", mode="max")
loss_fn = nn.CrossEntropyLoss(weight=torch.from_numpy(class_weights).to('cuda'))#nn.BCEWithLogitsLoss(reduction="none", )

In [7]:
dm = DataModule(data_dir='../../data', batch_size=cfg.batch_size)
model = SimpleCNN(cfg)
tmod = TrainModule(model, 
                   loss_fn=loss_fn, 
                   optimizer_name='Adam', 
                   optimizer_hparams={"lr": cfg.lr, "weight_decay": 1e-4},
                   cfg=cfg)
early_stop_callback = EarlyStopping(monitor="val_f1", patience=3, verbose=False, mode="max")

In [8]:
trainer = L.Trainer(
    #default_root_dir=
    max_epochs=20,
    accelerator="auto",
    devices="auto",
    enable_checkpointing=True,
    #overfit_batches=1,
    #profiler="simple",
    reload_dataloaders_every_n_epochs=False,
    logger=tb_logger,
    #callbacks=[early_stop_callback, lr_monitor, checkpoint_callback]
)
#trainer.fit(tmod, dm)

In [9]:
tuner = Tuner(trainer)
lr_finder = tuner.lr_find(model=tmod, datamodule=dm, min_lr=1e-7, max_lr=1e-2, num_training=100, attr_name="cfg")

In [10]:
# Results can be found in
print(lr_finder.results)

# Plot with
fig = lr_finder.plot(suggest=True)
fig.show()

# Pick point based on plot, or get suggestion
new_lr = lr_finder.suggestion()

In [14]:
new_lr

In [41]:
new_lr

In [11]:
new_lr